In [ ]:
class TabularDataset():
    "A `Numpy` dataset from a `TabularPandas` object"
    def __init__(self, to):
        self.cats = to.cats.to_numpy().astype(np.long)
        self.conts = to.conts.to_numpy().astype(np.float32)
        self.ys = to.ys.to_numpy()#.astype(np.int8) Remove as may be a regression problem?
    def __getitem__(self, idx):
        idx = idx[0]
        return self.cats[idx:idx+self.bs], self.conts[idx:idx+self.bs], self.ys[idx:idx+self.bs]
    def __len__(self): return len(self.cats)
    @property
    # Replaced self.y by self.ys (bug)
    def c(self): return 0 if self.ys is None else 1 if isinstance(self.ys[0], float) else len(np.unique(self.ys))

class TabDataLoader(DataLoader):
    # add **kwargs
    def __init__(self, dataset, bs=1, num_workers=0, device='cuda', shuffle=shuffle, **kwargs):
        "A `DataLoader` based on a `TabularDataset`"
        # Why always set shuffle to True?? 
        super().__init__(dataset, bs=bs, num_workers=num_workers, shuffle=True, device=device, drop_last=shuffle, **kwargs)
        dataset.bs = bs
        self.device, self.counter, self.shuffle = device, 0, shuffle
        self.dataset=dataset

    def create_item(self, s): return s
  
    def create_batch(self, b):
        cat, cont, y = self.dataset[b]
        return tensor(cat).to(self.device), tensor(cont).to(self.device), tensor(y).to(self.device)

    def get_idxs(self):
        idxs = Inf.count if self.indexed else Inf.nones
        if self.n is not None: idxs = list(range(len(self.dataset)))
        # Is this done once per epoch? If so, that's a good place to do it!!👍
        if self.shuffle: self.shuffle_fn()
        return idxs

    def shuffle_fn(self):
        "Shuffle dataset after each epoch"
        rng = np.random.permutation(len(self.dataset))
        self.dataset.cats = self.dataset.cats[rng]
        self.dataset.conts = self.dataset.conts[rng]
        self.dataset.ys = self.dataset.ys[rng]